<div style="background:#F5F7FA; height:100px; padding: 2em; font-size:14px;">
<span style="font-size:18px;color:#152935;">Want to do more?</span><span style="border: 1px solid #3d70b2;padding: 15px;float:right;margin-right:40px; color:#3d70b2; "><a href="https://ibm.co/wsnotebooks" target="_blank" style="color: #3d70b2;text-decoration: none;">Sign Up</a></span><br>
<span style="color:#5A6872;"> Try out this notebook with your free trial of IBM Watson Studio.</span>
</div>

# Finding Optimal Locations for 5 Portable Bike Shopsin San Francisco 


## Describe the business problem

* A bike shop/ repair company, "Forever Bike Industry" wants to gain visibility into challenges commuters encounter on the road and establish its name through the city by setting up five portable repairs shops (Vans) through the city of San Francisco.

* We implement a <a href="https://en.wikipedia.org/wiki/K-medians_clustering" target="_blank" rel="noopener noreferrer">K-Median model</a> to get the optimal location of our future shops.

## How  decision optimization can help

* Prescriptive analytics (decision optimization) technology recommends actions that are based on desired outcomes.  It takes into account specific scenarios, resources, and knowledge of past and current events. With this insight, your organization can make better decisions and has greater control of business outcomes.  

* Prescriptive analytics is the next step on the path to insight-based actions. It creates value through synergy with predictive analytics, which analyzes data to predict future outcomes.  

* Prescriptive analytics takes that insight to the next level by suggesting the optimal way to handle that future situation. Organizations that can act fast in dynamic conditions and make superior decisions in uncertain environments gain a strong competitive advantage.  
<br/>

__With prescriptive analytics, you can:__

* Automate the complex decisions and trade-offs to manage your limited resources better.
* Take advantage of a future opportunity or mitigate future risk.
* Proactively update recommendations based on changing events.
* Meet operational goals, increase customer loyalty, prevent threats and fraud, and optimize business processes.

## Use decision optimization

### Step 1: Download the library

Run the following code to install the __Decision Optimization CPLEX Modeling library__.  The *DOcplex* library contains the two modeling packages, _Mathematical Programming_ and _Constraint Programming_, referred to earlier.

In [2]:
import sys
try:
    import docplex.mp
except:
    if hasattr(sys, 'real_prefix'):
        #we are in a virtual env.
        !pip install docplex
    else:
        !pip install --user docplex

<i>__Note:__ that the more global package docplex contains another subpackage docplex.cp that is dedicated to Constraint Programming, another branch of optimization.</i>

### Step 2: Setup the Prescriptive Engine

* Subscribe to the [Decision Optimization on Cloud solve service](https://developer.ibm.com/docloud).
* Get the service URL and your personal API key (Client_id), and enter your credentials here:

In [3]:
url = "https://api-oaas.docloud.ibmcloud.com/job_manager/rest/v1/" #ENTER YOUR URL HERE
key = "api_d3ee51c7-5fc2-4bd7-a6c5-b63575bdad8d" #ENTER YOUR API KEY (CLIENT_id) HERE

### Step 3: Model the data

- The data for this problem is quite simple: it is composed of the list of Bike Counters and their geographical locations.
- The data is acquired from <a href="https://data.sfgov.org" target="_blank" rel="noopener noreferrer"> San Francisco  DataSF</a> as a JSON file, which is in the following format:
<code>
 [
      "row-arqb.js8w-kv65",
      "00000000-0000-0000-24B7-7C956AEB0BB8",
      0,
      1515693588,
      null,
      1515693588,
      null,
      "{ }",
      "POINT (-122.391252472708 37.758466406336)",
      "36",
      "38",
      "Indiana St North of 22nd St NB",
      "ROM",
      1479251097,
      "TWINTERS",
      1513711731,
      "37.758460999999997",
      "-122.391239",
      1467849600,
      null,
      "{F0CC02C2-0604-49FD-A261-7F912B6B826D}",
      "23619000",
      "7195000",
      "100018388"
    ]
</code>
This code snippet represents Bike Counter at "**Indiana St North of 22nd St NB**" located at **37.758, -122.391**


### Step 4: Prepare the data
We need to collect the list of all of the Bike Counters locations and keep their names, latitudes, and longitudes.

In [4]:
# Store longitude, latitude and street crossing name of each location.
class XPoint(object):
    def __init__(self, x, y):
        self.x = x
        self.y = y
    def __str__(self):
        return "P(%g_%g)" % (self.x, self.y)

class NamedPoint(XPoint):
    def __init__(self, name, x, y):
        XPoint.__init__(self, x, y)
        self.name = name
    def __str__(self):
        return self.name

#### Define how to compute the earth distance between 2 points
To easily compute the distance between 2 points, we use the Python package <a href="https://pypi.python.org/pypi/geopy" target="_blank" rel="noopener noreferrer">geopy</a>

In [5]:
try:
    import geopy.distance
except:
    if hasattr(sys, 'real_prefix'):
        #we are in a virtual env.
        !pip install geopy 
    else:
        !pip install --user geopy

In [6]:
# Simple distance computation between 2 locations.
from geopy.distance import great_circle
 
def get_distance(p1, p2):
    return great_circle((p1.y, p1.x), (p2.y, p2.x)).miles

#### Declare Function 
Parse the JSON file to get the list of public bike counters and store them as Python elements.

In [7]:
def build_bike_countrs_from_url(url, name_pos, lat_pos, long_pos):
    import requests
    import json

    r = requests.get(url)
    myjson = json.loads(r.text, parse_constant='utf-8')
    myjson = myjson['data']

    bike_counters = []
    k = 1
    for location in myjson:
        uname = location[name_pos] 
        try:
            latitude = float(location[lat_pos])
            longitude = float(location[long_pos])
        except TypeError:
            latitude = longitude = None
        try:
            name = uname
        except:
            name = "???"
        name = "P_%s_%d" % (name, k)
        if latitude and longitude:
            cp = NamedPoint(name, longitude, latitude)
            bike_counters.append(cp)
            k += 1
    return bike_counters

In [8]:
bike_counters = build_bike_countrs_from_url('https://data.sfgov.org/api/views/dcuu-jtmm/rows.json?accessType=DOWNLOAD',
                                   name_pos=11,
                                   lat_pos =16,
                                   long_pos =17)

In [9]:
print("There are %d public bike counters in San Francisco" % (len(bike_counters)))

There are 75 public bike counters in San Francisco


#### Define the number of Portable Bike Repair Shops 
Create a constant that indicates the number of Portable Bike Repair Shops we would like to open.

In [10]:
nb_shops = 5
print("The Forever Bike Industry would like to open %d Portable Bike Repair Shops " % nb_shops)

The Forever Bike Industry would like to open 5 Portable Bike Repair Shops 


#### Validate the data by displaying them
We will use the <a href="https://folium.readthedocs.org/en/latest/quickstart.html#getting-started" target="_blank" rel="noopener noreferrer">folium</a> library to display a map with markers.

In [11]:
try:
    import folium
except:
    if hasattr(sys, 'real_prefix'):
        #we are in a virtual env.
        !pip install folium 
    else:
        !pip install --user folium

In [12]:
import folium
map_osm = folium.Map(location=[37.6986, -122.409], zoom_start=11)
for bike_counter in bike_counters:
    lt = bike_counter.y
    lg = bike_counter.x
    folium.Marker([lt, lg]).add_to(map_osm)
map_osm

* After running the above code, the data is displayed, but it is impossible to determine where to ideally open portable repair bike shops by just looking at the map.

* Let's set up DOcplex to write and solve an optimization model that will help us to determine where to open portable repair bike shops an optimal way.


### Step 5: Set up the prescriptive model

In [13]:
from docplex.mp.environment import Environment
env = Environment()
env.print_information()

* system is: Darwin 64bit
* Python is present, version is 3.6.4
* docplex is present, version is (2, 7, 113)
* CPLEX wrapper is not available


#### Create the DOcplex model
The model contains all the business constraints and defines the objective.

In [14]:
from docplex.mp.model import Model
mdl = Model("Bike Shops")

#### Define the Decision Variables

In [15]:
BIGNUM = 999999999

# Ensure unique points
bike_counters = set(bike_counters)
# For simplicity, let's consider that bike shops candidate locations are the same as counters locations.
# That is: any counters location can also be selected as a portable bike shop location.
bikeshop_locations = bike_counters

# Decision vars
# Binary vars indicating which bike shop locations will be actually selected.
bikeshop_vars = mdl.binary_var_dict(bikeshop_locations, name="is_bikeshop")
#
# Binary vars representing the "assigned" counters for each portable bike shop location.
link_vars = mdl.binary_var_matrix(bikeshop_locations, bike_counters, "link")

__First constraint: if the distance is suspect, it needs to be excluded from the problem.__

In [16]:
for c_loc in bikeshop_locations:
    for b in bike_counters:
        if get_distance(c_loc, b) >= BIGNUM:
            mdl.add_constraint(link_vars[c_loc, b] == 0, "ct_forbid_{0!s}_{1!s}".format(c_loc, b))

__Second constraint: each _Bike Counters_ must be linked to a _Portable Bike Shop_ that is open.__
(for this demo no restrictions)

In [17]:
mdl.add_constraints(link_vars[c_loc, b] <= bikeshop_vars[c_loc]
                   for b in bike_counters
                   for c_loc in bikeshop_locations)
mdl.print_information()

Model: Bike Shops
 - number of variables: 5700
   - binary=5700, integer=0, continuous=0
 - number of constraints: 5625
   - linear=5625
 - parameters: defaults


__Third constraint: each _Bike Counters_ must be linked to exactly one _Portable Bike Shop.___ 

In [18]:
mdl.add_constraints(mdl.sum(link_vars[c_loc, b] for c_loc in bikeshop_locations) == 1
                   for b in bike_counters)
mdl.print_information()

Model: Bike Shops
 - number of variables: 5700
   - binary=5700, integer=0, continuous=0
 - number of constraints: 5700
   - linear=5700
 - parameters: defaults


__Fourth constraint: there is a fixed number of _Portable Bike Shop_ to open.__

In [19]:
# Total nb of bike shops
mdl.add_constraint(mdl.sum(bikeshop_vars[c_loc] for c_loc in bikeshop_locations) == nb_shops)

# Print model information
mdl.print_information()

Model: Bike Shops
 - number of variables: 5700
   - binary=5700, integer=0, continuous=0
 - number of constraints: 5701
   - linear=5701
 - parameters: defaults


__Express the objective__
* The objective is to minimize the total distance between Bike Counters and Portable Bike Shop so that "forever Bike Industry can strategically place their portable bike shops around San Francisco.

In [20]:
# Minimize total distance from points to hubs
total_distance = mdl.sum(link_vars[c_loc, b] * get_distance(c_loc, b) for c_loc in bikeshop_locations for b in bike_counters)
mdl.minimize(total_distance)

In [21]:
print("# Bike Counter locations: %d" % len(bikeshop_locations))
print("# Portable Bike Shops: %d" % nb_shops)

assert mdl.solve(url=url, key=key), "!!! Solve of the model fails"

# Bike Counter locations: 75
# Portable Bike Shops: 5


In [22]:
total_distance = mdl.objective_value
open_bikeshops = [c_loc for c_loc in bikeshop_locations if bikeshop_vars[c_loc].solution_value == 1]
not_bikeshops = [c_loc for c_loc in bikeshop_locations if c_loc not in open_bikeshops]
edges = [(c_loc, b) for b in bike_counters for c_loc in bikeshop_locations if int(link_vars[c_loc, b]) == 1]

print("Total distance: %g" % total_distance)
print("# Portable Bike Shops: {0}".format(len(open_bikeshops)))
for c in open_bikeshops:
    print("Suggested location for Portable Bike Shop: {0!s}".format(c))

Total distance: 66.8264
# Portable Bike Shops: 5
Suggested location for Portable Bike Shop: P_Market St. Totem N WB_14
Suggested location for Portable Bike Shop: P_Kirkham west of 7th Avenue_57
Suggested location for Portable Bike Shop: P_Potrero south of 23rd NB_66
Suggested location for Portable Bike Shop: P_Holloway Ave West of Stratford EB_17
Suggested location for Portable Bike Shop: P_Baker south of Golden Gate_23


In [23]:
import folium
map_osm = folium.Map(location=[37.768, -122.409], zoom_start=12)
for bikeshop in open_bikeshops:
    lt = bikeshop.y
    lg = bikeshop.x
    folium.Marker([lt, lg], icon=folium.Icon(color='green',icon='info-sign')).add_to(map_osm)
    
for b in bike_counters:
    if b not in open_bikeshops:
        lt = b.y
        lg = b.x
        folium.Marker([lt, lg]).add_to(map_osm)
    

for (c, b) in edges:
    coordinates = [[c.y, c.x], [b.y, b.x]]
    map_osm.add_child(folium.PolyLine(coordinates, color='#FF0000', weight=5))

map_osm

## Summary

From our analysis, we can see the optimum locations for our Portable Bike Shops, even though we use a straight line to calculate the distance between Bike Counters and Portable Bike shops. The City of San Francisco is made up of a grid system which makes it easy to navigate and with locations strategically placed around helps to deal with an inflow of work. 

* Potrero south of 23rd NB
* Holloway Ave West of Stratford EB
* Market St. Totem N WB
* Baker south of Golden Gate
* Kirkham west of 7th Avenue